# Run stock-price prediction model

## Notebook ([`papermill`](https://papermill.readthedocs.io/en/latest/index.html)) parameters

In [1]:
run_time = '2019-07-10T11:23'
stop_minutes = 10
dir = None
earliest_data_cutoff = '2019-04-01'
ticker = 'AAPL'
batch_size = 100000
max_epochs = 10000

## Date/Time helpers

In [3]:
from sys import executable as python
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()

if run_time:
    run_time = parse(run_time)
else:
    run_time = dt.now()

run_date = run_time.date()

if stop_minutes:
    stop_at = run_time + Δ(minutes=stop_minutes)
else:
    stop_at = None

earliest_data_cutoff = parse(earliest_data_cutoff)
earliest_data_cutoff_date = earliest_data_cutoff.date()
    
print('Running at %s (stopping within %dmins)' % (run_time.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes))

Running at 2019-07-10 11:23:00 (stopping within 10mins)


## Initialize `data`, `models` directories

In [4]:
from pathlib import Path
if not dir:
    dir = Path.cwd()

data_dir = dir / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
models_dir = dir / 'models'

## Read IEX API credentials
Search for a secret key in:
- `$IEX_SECRET_KEY` env var
- `iex.secret_key` in a config file located at:
  - `$CONFIG_PATH` if set
  - `~/.config/iex.ini` otherwise

In [5]:
from os import environ as env

api = 'https://cloud.iexapis.com'

SECRET_KEY_VAR = 'IEX_SECRET_KEY'
CONFIG_PATH_VAR = 'CONFIG_PATH'

if SECRET_KEY_VAR in env:
    secret_key = env[SECRET_KEY_VAR]
else:
    if CONFIG_PATH_VAR in env:
        config_path = Path(env[CONFIG_PATH_VAR])
    else:
        config_path = Path.home() / '.config' / 'iex.ini'

    if not config_path.exists():
        raise Exception("No %s env var, and config path %s doesn't exist" % (SECRET_KEY_VAR, config_path))

    from configparser import ConfigParser
    config = ConfigParser()
    config.read(str(config_path))
    iex_config = config['iex']
    secret_key = iex_config['secret_key']

## Initialize `dates` and `minutes` that trading occurs during

In [6]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = earliest_data_cutoff_date
end_date = run_date + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 73 days ([2019-04-01,2019-07-11)), 390 minutes per day


In [7]:
def get_ticker_dir(ticker):
    return data_dir / ticker

def get_ticker_date_path(ticker, date):
    return get_ticker_dir(ticker) / date.strftime('%Y%m%d')

In [8]:
!{python} -m pip install -Uq requests
from requests import get as GET

In [9]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [10]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [11]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
); None

CPU times: user 403 ms, sys: 240 ms, total: 643 ms
Wall time: 6.02 s


In [12]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [13]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [14]:
def load_ticker_date_df(date, ticker, latest_data_datetime=None):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    if latest_data_datetime:
        df = df.loc[df.datetime <= latest_data_datetime]
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [15]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker, latest_data_datetime=run_time) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [16]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 2.06 s, sys: 40.7 ms, total: 2.1 s
Wall time: 2.14 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.60,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.98,190.980,190.64,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.93,191.090,190.78,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.83,191.010,190.76,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.70,190.760,190.60,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-07-10 11:19:00,202.280,202.27,202.325,202.22,202.272,1408,284798.900,18
2019-07-10 11:20:00,202.260,202.26,202.340,202.17,202.241,2601,526028.755,28
2019-07-10 11:21:00,202.310,202.36,202.360,202.20,202.269,1670,337789.600,20


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [17]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [18]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [19]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
2019-04-18 14:27:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,203.820
2019-04-18 14:44:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,203.920
2019-04-24 13:47:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.570
2019-04-30 12:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,200.830


Spot-check: consecutive pairs of minutes where no trades occurred

In [20]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [21]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              24474
close             24474
high              24474
low               24474
average           24474
volume            24501
notional          24501
numberOfTrades    24501
prev_close        24501
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [22]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

open              24501
close             24501
high              24501
low               24501
average           24501
volume            24501
notional          24501
numberOfTrades    24501
prev_close        24501
dtype: int64

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [23]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
2019-04-05 14:29:00,196.795,196.795,196.795,196.795,196.795,0,0.000,0,196.795
2019-04-15 13:32:00,198.920,198.910,198.920,198.910,198.917,400,79567.000,4,198.910
2019-04-15 13:33:00,198.910,198.910,198.910,198.910,198.910,0,0.000,0,198.910
2019-04-16 13:38:00,199.730,199.725,199.740,199.710,199.727,1104,220498.920,12,199.700


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [24]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 14:30:00,0.000076,0.000051,0.000051,-0.000102,-0.000056
2019-04-15 13:33:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-15 13:34:00,0.000050,0.000151,0.000151,-0.000025,0.000050
2019-04-16 13:39:00,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
lg.count()

open              24501
close             24501
high              24501
low               24501
average           24501
volume            24501
notional          24501
numberOfTrades    24501
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [26]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-07-10 11:19:00,-0.000049,-0.000049,0.000222,-0.000297,-0.000040,4.242641,4.372257
2019-07-10 11:20:00,-0.000049,0.000000,0.000395,-0.000445,-0.000094,5.291503,4.542154
2019-07-10 11:21:00,0.000247,0.000247,0.000247,-0.000544,-0.000203,4.472136,4.436752


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [27]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-07-10 11:19:00,-0.000049,-0.000049,0.000222,-0.000297,-0.000040,4.242641,4.372257,-0.000049
2019-07-10 11:20:00,-0.000049,0.000000,0.000395,-0.000445,-0.000094,5.291503,4.542154,0.000494
2019-07-10 11:21:00,0.000247,0.000247,0.000247,-0.000544,-0.000203,4.472136,4.436752,-0.000148


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [28]:
lg.dropna(how='any', inplace=True)
lg.count()

open          24437
close         24437
hi            24437
lo            24437
avg           24437
trades        24437
avgVol        24437
next_close    24437
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [29]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
n = tn + vn
tf, vf = tn / n, vn / n
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

19487 training samples (79.74%), 4950 validation (20.26%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [30]:
lg_t = lg[~vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,1.143325e-05,0.000206
close,-8.813123e-06,0.000608
hi,3.084123e-04,0.000431
lo,-3.259922e-04,0.000480
avg,-2.011631e-06,0.000382
trades,4.205853e+00,1.658067
avgVol,4.514693e+00,0.338751
next_close,8.208708e-07,0.000634


Helpers for showing descriptive statistics of each column of a DataFrame:

In [31]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [32]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,24437,0.000012,0.000206,0.021302,5.350489,-0.002417,0.001974
close,24437,-0.000007,0.000601,-1.253593,52.295821,-0.018992,0.007019
hi,24437,0.000308,0.000430,3.727550,26.126146,0.000000,0.007200
lo,24437,-0.000324,0.000468,-8.704299,292.593904,-0.023854,0.000000
avg,24437,-0.000001,0.000375,-2.683808,119.264891,-0.015078,0.004867
trades,24437,4.203664,1.658853,1.154413,2.718264,0.000000,17.860571
avgVol,24437,4.515110,0.337442,-2.674086,44.862953,0.000000,7.509961
next_close,24437,0.000003,0.000630,-1.303242,43.690752,-0.018822,0.006486


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [33]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.055474,-8.541092,-0.716231,-10.707398,-6.223005,1.463992,0.242837,2.725435
2019-04-01 09:31:00,1.216843,2.425959,2.691065,0.023558,0.841425,0.875117,0.338182,-0.414132
2019-04-01 09:32:00,1.976557,-1.104219,-0.351612,-2.375229,-1.486721,0.766780,-0.539172,-0.827294
2019-04-01 09:33:00,1.214977,-1.276936,-0.351459,-1.721409,-0.460233,0.597262,-0.247605,-1.075742
2019-04-01 09:34:00,-1.835597,-0.502557,-0.716231,-1.068528,-1.283376,-0.447359,-0.041205,1.238388
...,...,...,...,...,...,...,...,...
2019-07-10 11:18:00,-1.254530,-0.066762,0.431442,0.370024,-0.007664,0.227206,1.879462,-0.157184
2019-07-10 11:19:00,-0.295333,-0.066770,-0.199657,0.061078,-0.098142,0.022187,-0.420475,-0.079245
2019-07-10 11:20:00,-0.295356,0.014485,0.202134,-0.247976,-0.240351,0.654769,0.081066,0.778039


In [34]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,24437,0.001074,0.999003,0.021302,5.350489,-11.781984,9.523460
close,24437,0.002344,0.987300,-1.253593,52.295821,-31.199835,11.551574
hi,24437,-0.000511,0.998699,3.727550,26.126146,-0.716231,16.005099
lo,24437,0.003831,0.975599,-8.704299,292.593904,-48.996730,0.678864
avg,24437,0.002334,0.980965,-2.683808,119.264891,-39.415539,12.730666
trades,24437,-0.001320,1.000474,1.154413,2.718264,-2.536601,8.235326
avgVol,24437,0.001231,0.996137,-2.674086,44.862953,-13.327468,8.842093
next_close,24437,0.003372,0.992674,-1.303242,43.690752,-29.676610,10.224877


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [35]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.055474,-8.541092,-0.716231,-10.707398,-6.223005,1.463992,0.242837
2019-04-01 09:31:00,1.216843,2.425959,2.691065,0.023558,0.841425,0.875117,0.338182
2019-04-01 09:32:00,1.976557,-1.104219,-0.351612,-2.375229,-1.486721,0.766780,-0.539172
2019-04-01 09:33:00,1.214977,-1.276936,-0.351459,-1.721409,-0.460233,0.597262,-0.247605
2019-04-01 09:34:00,-1.835597,-0.502557,-0.716231,-1.068528,-1.283376,-0.447359,-0.041205
...,...,...,...,...,...,...,...
2019-07-10 11:18:00,-1.254530,-0.066762,0.431442,0.370024,-0.007664,0.227206,1.879462
2019-07-10 11:19:00,-0.295333,-0.066770,-0.199657,0.061078,-0.098142,0.022187,-0.420475
2019-07-10 11:20:00,-0.295356,0.014485,0.202134,-0.247976,-0.240351,0.654769,0.081066


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [36]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(24437, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [37]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(19487, 30, 7), (19487,), (4950, 30, 7), (4950,), (24437, 30, 7), (24437,)]

In [38]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.60,191.189,4320,825935.940,44,191.645,190.98,1.001731
2019-04-01 09:31:00,190.700,190.98,190.980,190.64,190.761,3246,619210.510,32,190.650,190.93,0.999738
2019-04-01 09:32:00,191.060,190.93,191.090,190.78,190.951,2253,430211.740,30,190.980,190.83,0.999476
2019-04-01 09:33:00,190.980,190.83,191.010,190.76,190.946,2241,427911.290,27,190.930,190.70,0.999319
2019-04-01 09:34:00,190.760,190.70,190.760,190.60,190.666,1069,203822.465,12,190.830,190.85,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-07-10 11:19:00,202.280,202.27,202.325,202.22,202.272,1408,284798.900,18,202.290,202.26,0.999951
2019-07-10 11:20:00,202.260,202.26,202.340,202.17,202.241,2601,526028.755,28,202.270,202.36,1.000494
2019-07-10 11:21:00,202.310,202.36,202.360,202.20,202.269,1670,337789.600,20,202.260,202.33,0.999852


## Build model

In [39]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [40]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(window, len(cols))))
model.add(Dense(1))
model.build()
model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [41]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.6580881103294846, 0.6459524795406989)

In [42]:
def diffs_df(model):
    def stats(x, y, name=None, predict=True):
        suffix = ' (%s)' % name if name else ''
        if predict:
            predx = model.predict(x)[:, 0]
        else:
            predx = x

        diffs = (y - predx)
        mae = abs(diffs)
        mse = mae**2

        return DF({
            ('y^' + suffix): predx,
            ('Δ' + suffix): diffs,
            ('|Δ|' + suffix): mae,
            ('Δ²' + suffix): mse,
        })

    train = stats(tx, ty, 't')
    val = stats(vx, vy, 'v')
    predx_t = train['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ])
    all = stats(predx, np.concatenate([ ty, vy ]), name='*', predict=False)

    return concat([
        desc(train),
        desc(val),
        desc(all)
    ])

In [43]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),19487,0.003941,0.397024,-0.025626,-1.265110,-7.747461e-01,0.739245
Δ (t),19487,-0.003941,1.079284,-1.001440,36.922616,-2.969386e+01,9.607805
|Δ| (t),19487,0.751191,0.774953,5.485647,119.460632,4.862307e-05,29.693862
Δ² (t),19487,1.164810,7.268473,94.389727,11135.373339,2.364203e-09,881.725423
y^ (v),4950,0.012741,0.399142,-0.065032,-1.277423,-7.650745e-01,0.738222
Δ (v),4950,0.003904,1.050895,-0.147368,9.561105,-1.185861e+01,10.842103
|Δ| (v),4950,0.740874,0.745245,3.398056,26.236780,3.135688e-04,11.858606
Δ² (v),4950,1.104172,3.754346,20.104001,611.828160,9.832537e-08,140.626540
y^ (*),24437,0.005724,0.397462,-0.033592,-1.267988,-7.747461e-01,0.739245
Δ (*),24437,-0.002352,1.073577,-0.839554,31.856985,-2.969386e+01,10.842103


In [46]:
model_ckpt_path = models_dir / run_time.strftime('%Y-%m-%dT%H:%M.ckpt'); model_ckpt_path

PosixPath('/Users/ryan/c/pipelines/notebooks/models/2019-07-10T11:23.ckpt')

In [47]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_best_only=True, period=20),
          ]
         )
diffs_df(model)

Train on 19487 samples, validate on 4950 samples
Epoch 1/10000
19487/19487 [==============================] - 1s 70us/sample - loss: 0.7512 - val_loss: 0.7396
Epoch 2/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.7500 - val_loss: 0.7384
Epoch 3/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.7488 - val_loss: 0.7372
Epoch 4/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.7476 - val_loss: 0.7360
Epoch 5/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.7464 - val_loss: 0.7348
Epoch 6/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.7452 - val_loss: 0.7336
Epoch 7/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.7441 - val_loss: 0.7324
Epoch 8/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.7429 - val_loss: 0.7312
Epoch 9/10000
19487/19487 [==============================] - 0s 6us/sample - l

Epoch 68/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.7000 - val_loss: 0.6886
Epoch 69/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6996 - val_loss: 0.6881
Epoch 70/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6992 - val_loss: 0.6877
Epoch 71/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6988 - val_loss: 0.6873
Epoch 72/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6984 - val_loss: 0.6868
Epoch 73/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6979 - val_loss: 0.6864
Epoch 74/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6975 - val_loss: 0.6860
Epoch 75/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6971 - val_loss: 0.6856
Epoch 76/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6967 - val_loss: 0.6851
Epoch 77/1

Epoch 140/10000
INFO:tensorflow:Assets written to: /Users/ryan/c/pipelines/notebooks/models/2019-07-10T11:23.ckpt/assets
19487/19487 [==============================] - 2s 86us/sample - loss: 0.6763 - val_loss: 0.6635
Epoch 141/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6761 - val_loss: 0.6632
Epoch 142/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6759 - val_loss: 0.6630
Epoch 143/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6756 - val_loss: 0.6627
Epoch 144/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6754 - val_loss: 0.6625
Epoch 145/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6752 - val_loss: 0.6622
Epoch 146/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6749 - val_loss: 0.6620
Epoch 147/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6747 - val_loss: 0.6617
Epoch 

Epoch 210/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6654 - val_loss: 0.6516
Epoch 211/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6653 - val_loss: 0.6515
Epoch 212/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6652 - val_loss: 0.6515
Epoch 213/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6651 - val_loss: 0.6514
Epoch 214/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6650 - val_loss: 0.6513
Epoch 215/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6649 - val_loss: 0.6512
Epoch 216/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6649 - val_loss: 0.6511
Epoch 217/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6648 - val_loss: 0.6510
Epoch 218/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6647 - val_loss: 0.6509
E

19487/19487 [==============================] - 2s 86us/sample - loss: 0.6613 - val_loss: 0.6472
Epoch 281/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6612 - val_loss: 0.6471
Epoch 282/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6612 - val_loss: 0.6471
Epoch 283/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6612 - val_loss: 0.6470
Epoch 284/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6611 - val_loss: 0.6470
Epoch 285/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6611 - val_loss: 0.6470
Epoch 286/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6611 - val_loss: 0.6469
Epoch 287/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6610 - val_loss: 0.6469
Epoch 288/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6610 - val_loss: 0.6469
Epoch 289/10000


...

Epoch 493/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 494/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 495/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 496/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 497/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 498/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 499/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 500/10000
INFO:tensorflow:Assets written to: /Users/ryan/c/pipelines/notebooks/models/2019-07-10T11:23.ckpt/assets
19487/19487 [==============================] - 1s 73us/sample - loss: 0.6581 - val_loss: 0.6442
Epoch 

Epoch 563/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 564/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 565/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 566/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 567/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 568/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 569/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6579 - val_loss: 0.6441
Epoch 570/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6578 - val_loss: 0.6441
Epoch 571/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6578 - val_loss: 0.6441
E

Epoch 634/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 635/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 636/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 637/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 638/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 639/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 640/10000
INFO:tensorflow:Assets written to: /Users/ryan/c/pipelines/notebooks/models/2019-07-10T11:23.ckpt/assets
19487/19487 [==============================] - 1s 69us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 641/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6577 - val_loss: 0.6440
Epoch 

Epoch 706/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6576 - val_loss: 0.6441
Epoch 707/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6576 - val_loss: 0.6441
Epoch 708/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6576 - val_loss: 0.6441
Epoch 709/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6575 - val_loss: 0.6441
Epoch 710/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6575 - val_loss: 0.6441
Epoch 711/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6575 - val_loss: 0.6441
Epoch 712/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6575 - val_loss: 0.6441
Epoch 713/10000
19487/19487 [==============================] - 0s 6us/sample - loss: 0.6575 - val_loss: 0.6441
Epoch 714/10000
19487/19487 [==============================] - 0s 7us/sample - loss: 0.6575 - val_loss: 0.6441
E

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),19487,0.018910,0.059857,0.746404,3.031576,-3.136145e-01,0.446048
Δ (t),19487,-0.018910,0.999745,-1.575435,51.609013,-2.993859e+01,9.271206
|Δ| (t),19487,0.657662,0.753197,6.073610,140.513520,4.907209e-07,29.938594
Δ² (t),19487,0.999796,7.326125,97.097530,11544.577972,2.408070e-13,896.319397
y^ (v),4950,0.018358,0.060280,0.556210,2.196864,-2.775629e-01,0.317561
Δ (v),4950,-0.001714,0.961199,-0.499012,12.962289,-1.207749e+01,10.142740
|Δ| (v),4950,0.644028,0.713478,3.711976,30.634679,3.351559e-04,12.077490
Δ² (v),4950,0.923720,3.573829,22.286090,736.921979,1.123295e-07,145.865759
y^ (*),24437,0.018798,0.059942,0.707164,2.859875,-3.136145e-01,0.446048
Δ (*),24437,-0.015427,0.992063,-1.378265,44.772713,-2.993859e+01,10.142740


In [48]:
hist = DF(model.history.history)
hist.iloc[-110:-90]

,loss,val_loss
636,0.657682,0.644029
637,0.657680,0.644030
638,0.657678,0.644030
639,0.657676,0.644030
640,0.657674,0.644030
641,0.657672,0.644030
642,0.657670,0.644030
643,0.657668,0.644029
644,0.657666,0.644029
645,0.657664,0.644028


In [ ]:
hist.iloc[-200:]

In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)